In [18]:
import pandas as pd
import glob

In [19]:
# Step 1: Read the CSV file
df = pd.read_csv('author&funding_info.csv')

# Step 2: Create a new column `expiration_year`
df['expiration_year'] = df['expiration_date'].str[-4:]

# Step 3: Keep specified columns only
columns_to_keep = [
    'email', 'institution', 'interests', 'award_year', 'expiration_year',
    'award_amount', 'award_title', 'total_citations', 'h_index',
    'citation_2011', 'citation_2012', 'citation_2013', 'citation_2014',
    'citation_2015', 'citation_2016', 'citation_2017', 'citation_2018',
    'citation_2019', 'citation_2020'
]

# Ensure all specified columns exist in DataFrame to avoid KeyError
columns_to_keep = [col for col in columns_to_keep if col in df.columns]

# Redefine DataFrame to only include specified columns
df = df[columns_to_keep]
df.head()

,email,institution,interests,award_year,expiration_year,award_amount,award_title,total_citations,h_index,citation_2011,citation_2012,citation_2013,citation_2014,citation_2015,citation_2016,citation_2017,citation_2018,citation_2019,citation_2020
0,talalasad@earthlink.net,CUNY Graduate School University Center,['organic chemistry'],2011,2012,19901,Doctoral Dissertation Research: Mental Disorde...,106,3,2.0,6.0,4.0,6.0,12.0,13.0,10.0,11.0,13.0,5.0
1,vanboven@colorado.edu,University of Colorado at Boulder,"['Social psychology', 'judgment and decision m...",2011,2015,250000,EAGER: Perceiving Political Distributions,12916,46,363.0,523.0,600.0,621.0,745.0,759.0,782.0,921.0,868.0,1234.0
2,bedricks@ohsu.edu,Oregon Health & Science University,"['NLP', 'Medical Informatics']",2011,2013,114140,"Corpora of Non-Linguistic Symbol Systems, and ...",2230,21,63.0,54.0,76.0,74.0,84.0,70.0,87.0,105.0,153.0,253.0
3,ost@psu.edu,Pennsylvania State Univ University Park,"['Archaeology (eastern North America', 'Warfar...",2011,2015,113625,Late Prehistoric Warfare in Eastern North Amer...,9549,42,295.0,366.0,379.0,458.0,425.0,479.0,582.0,601.0,563.0,633.0
4,gable@psych.ucsb.edu,University of California-Santa Barbara,NaN,2011,2015,377951,Capitalizing on Positive Events,30300,54,1119.0,1414.0,1563.0,1701.0,2017.0,1811.0,2022.0,2133.0,2204.0,2186.0


In [20]:
# Expand each author to 10 rows, adding a year column
expanded_rows = []
for _, row in df.iterrows():
    for year in range(2011, 2021):
        new_row = row.copy()
        new_row['year'] = year
        # Update award_amount
        if year >= int(row['award_year']) and year <= int(row['expiration_year']):
            new_row['award_amount'] = row['award_amount']
        else:
            new_row['award_amount'] = 0
        expanded_rows.append(new_row)

expanded_df = pd.DataFrame(expanded_rows)

# Merge award amounts for duplicate authors
grouped_df = expanded_df.groupby(['email', 'year'], as_index=False).agg({
    'institution': 'first',
    'interests': 'first',
    'award_year': 'first',
    'expiration_year': 'first',
    'award_amount': 'sum',
    'award_title': 'first',
    'total_citations': 'first',
    'h_index': 'first',
    **{f'citation_{year}': 'first' for year in range(2011, 2021)}
})

# Create citation column
for year in range(2011, 2021):
    grouped_df.loc[grouped_df['year'] == year, 'citation'] = grouped_df[f'citation_{year}']

# Drop citation_XXXX columns that are no longer needed
citation_columns = [f'citation_{year}' for year in range(2011, 2021)]
grouped_df.drop(columns=citation_columns, inplace=True)

grouped_df.head()

,email,year,institution,interests,award_year,expiration_year,award_amount,award_title,total_citations,h_index,citation
0,00t0holtgrav@bsu.edu,2011,Ball State University,['Social Psychology and Language'],2012,2017,0,The Role of Interpersonal Processes in the Int...,10066,46,367.0
1,00t0holtgrav@bsu.edu,2012,Ball State University,['Social Psychology and Language'],2012,2017,165000,The Role of Interpersonal Processes in the Int...,10066,46,409.0
2,00t0holtgrav@bsu.edu,2013,Ball State University,['Social Psychology and Language'],2012,2017,165000,The Role of Interpersonal Processes in the Int...,10066,46,534.0
3,00t0holtgrav@bsu.edu,2014,Ball State University,['Social Psychology and Language'],2012,2017,165000,The Role of Interpersonal Processes in the Int...,10066,46,550.0
4,00t0holtgrav@bsu.edu,2015,Ball State University,['Social Psychology and Language'],2012,2017,165000,The Role of Interpersonal Processes in the Int...,10066,46,496.0


In [21]:
# Optionally, save this expanded dataframe
# grouped_df.to_csv('author&funding_info_expanded.csv', index=False)

In [23]:
# # Define the folder and pattern to search for
# folder_path = 'publist/*.csv'

# # Use glob to find all csv files
# csv_files = glob.glob(folder_path)

# # Initialize an empty DataFrame to hold the merged data
# merged_df = pd.DataFrame()

# # Loop through the found files
# for file in csv_files:
#     # Read the CSV file
#     df = pd.read_csv(file, encoding='utf-8-sig')
#     # Check and merge the data
#     merged_df = pd.concat([merged_df, df], ignore_index=True)

# # Filter rows where the Year column is between 2011 and 2020
# filtered_df = merged_df[(merged_df['Year'] >= 2011) & (merged_df['Year'] <= 2020)]

# # Save the final DataFrame to a CSV file
# filtered_df.to_csv('merged_publications_2011_2020.csv', index=False, encoding='utf-8-sig')
filtered_df = pd.read_csv('merged_publications_2011_2020.csv')

In [25]:
df = pd.read_csv('author&funding_info.csv', encoding='utf-8-sig')
# Step 2: Extract the user parameter from the Paper URL column
# Use apply and lambda function to extract the user parameter from the URL
filtered_df['user'] = filtered_df['Paper URL'].apply(lambda x: x.split('&')[2].split('=')[1])

# Step 3: Find the corresponding user in author_info_2016 and get email information
# Create a dictionary to map user to email
user_to_email = df.set_index('url')['email'].to_dict()

# Step 4: Add the email information to a new column in pub_info_2013_2019
# Use the map function and the dictionary just created to map user to email
filtered_df['email'] = filtered_df['user'].map(lambda x: user_to_email.get(f"https://scholar.google.com/citations?hl=en&user={x}"))
pub_info_df = filtered_df.drop(columns=['user'])

# Optional: Save the updated filtered_df to CSV
# pub_info_df.to_csv('updated_filtered_df.csv', index=False, encoding='utf-8-sig')

In [26]:
# Remove duplicate rows where both Title and email columns are the same
unique_df = pub_info_df.drop_duplicates(subset=['Title', 'email'], keep='first')

# Optional: Save the updated DataFrame to CSV
# unique_df.to_csv('unique_filtered_df.csv', index=False, encoding='utf-8-sig')

In [8]:
# Initialize new columns
grouped_df['pub_num'] = 0
grouped_df['top_cited'] = 0.0

# Iterate through each row in grouped_df
for index, row in grouped_df.iterrows():
    # Find matching rows in unique_df
    matches = unique_df[(unique_df['email'] == row['email']) & (unique_df['Year'] == row['year'])]
    
    # Update the pub_num column
    grouped_df.at[index, 'pub_num'] = len(matches)
    
    # If there are matching rows, calculate top_cited
    if len(matches) > 0:
        # Sort the 'Cited by' column in descending order and take the mean of the top three
        top_cited_avg = matches['Cited by'].nlargest(3).mean() if len(matches) >= 3 else matches['Cited by'].mean()
        grouped_df.at[index, 'top_cited'] = top_cited_avg
    else:
        # If there are no matching rows, top_cited is 0
        grouped_df.at[index, 'top_cited'] = 0

# Save the final regression_data used for later regression model fitting
grouped_df.to_csv('regression_data.csv', index=False, encoding='utf-8-sig')